Cross-Validation

1) What is the logic behind cross-validation? Is it always necessary? What problems does it solve? Which doesn’t it solve?

The goal is to estimate the expected level of fit of a model to a dataset that's indpendent of the data that is uesd to train the model.

2) Return to last week’s analysis of your data with naive Bayes. Using the e1071 library, check the model that you fit with cross-validation. Interpret the results.

Preformed using sklearn, results below.

SVM

3) Using the same labels that you used last week, fit a SVM on the same training set and cross validate. How do the results compare to those with naive Bayes? Look at the labels assigned to different observations in the test set. Is there any pattern?

Naive bayes resulted in a ~58% sussccessful classification and the SVM closely followed with ~56%. Both models used the same random seed (42).  

# Setup

In [11]:
import numpy as np
import pandas as pd

from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Load Data

In [2]:
df_game_reviews = pd.read_csv(r'../data/processed/game_reviews_processed.csv', low_memory=False)

In [3]:
df_game_reviews.columns

Index(['recommendationid', 'language', 'Valence Intensity', 'Joy Intensity',
       'Anger Intensity', 'Fear Intensity', 'Sadness Intensity', 'Sentiment',
       'Emotion', 'review',
       ...
       'Unnamed: 93', 'Unnamed: 94', 'Unnamed: 95', 'Unnamed: 96',
       'Unnamed: 97', 'Unnamed: 98', 'Unnamed: 99', 'Unnamed: 100',
       'Unnamed: 101', 'Unnamed: 102'],
      dtype='object', length=103)

In [4]:
# clean
# subset data that's hand-coded
def prune_cols(df):
    # keep only needed cols and subset english only
    global df_game_reviews
    df_game_reviews = df_game_reviews.loc[df_game_reviews['Sentiment'].notnull(), ['recommendationid', 'Sentiment', 'Emotion', 'review']] # subsets for english
    df_game_reviews = df_game_reviews.loc[df_game_reviews['review'].notnull(), ['recommendationid', 'Sentiment', 'Emotion', 'review']] # subsets for NaN reviews   
    return df_game_reviews

prune_cols(df_game_reviews)

,recommendationid,Sentiment,Emotion,review
0,71182846,Negative,Anger,"When I just joined the battle, my squad was at..."
1,71182712,Negative,Sadness,Game is fun and exciting when it works. Very s...
5,26578229,Positive,Joy,Excellent game. \r\n\r\nPlenty of fun to be ha...
7,71178247,Negative,Anger,Players are too serious and you can't have fun...
8,71177738,Positive,Joy,great simulation game
...,...,...,...,...
472,70608305,Positive,Joy,the best\r\n
473,70608216,Positive,Joy,"Most fun when people work together, it is what..."
474,70606979,Negative,Anger,"The game has a lot to learn, but being able to..."
476,70606193,Positive,Joy,"9/10 would recommend, bring friends for 10/10"


In [5]:
# clean
# one hot encode emotions into new cols

def code_cols(df):
    df['emotion_code'] = None
    df['emotion_joy'] = None
    df['emotion_fear'] = None
    df['emotion_anger'] = None
    df['emotion_sadness'] = None
    
    for index, row in df.iterrows():
        if  df.loc[index,'Emotion'] == 'Joy':
            df.loc[index,'emotion_code'] = 1
        elif df.loc[index,'Emotion'] == 'Anger':
            df.loc[index,'emotion_code'] = 2
        elif df.loc[index,'Emotion'] == 'Fear':
            df.loc[index,'emotion_code'] = 3
        else:
            df.loc[index,'emotion_code'] = 4
        
    
    for index, row in df.iterrows():
        if  df.loc[index,'Emotion'] == 'Joy':
            df.loc[index,'emotion_joy'] = 1
            df.loc[index,'emotion_fear'] = 0
            df.loc[index,'emotion_anger'] = 0
            df.loc[index,'emotion_sadness'] = 0
        elif df.loc[index,'Emotion'] == 'Anger':
            df.loc[index,'emotion_joy'] = 0
            df.loc[index,'emotion_fear'] = 0
            df.loc[index,'emotion_anger'] = 1
            df.loc[index,'emotion_sadness'] = 0
        elif df.loc[index,'Emotion'] == 'Fear':
            df.loc[index,'emotion_joy'] = 0
            df.loc[index,'emotion_fear'] = 1
            df.loc[index,'emotion_anger'] = 0
            df.loc[index,'emotion_sadness'] = 0
        else:
            df.loc[index,'emotion_joy'] = 0
            df.loc[index,'emotion_fear'] = 0
            df.loc[index,'emotion_anger'] = 0
            df.loc[index,'emotion_sadness'] = 1
    
    #set dtype()
    df['emotion_code'] = df['emotion_code'].astype('int64')    
    df['emotion_joy'] = df['emotion_joy'].astype('int64')
    df['emotion_fear'] = df['emotion_fear'].astype('int64')
    df['emotion_anger'] = df['emotion_anger'].astype('int64')
    df['emotion_sadness'] = df['emotion_sadness'].astype('int64')
        
    return df

code_cols(df_game_reviews)

,recommendationid,Sentiment,Emotion,review,emotion_code,emotion_joy,emotion_fear,emotion_anger,emotion_sadness
0,71182846,Negative,Anger,"When I just joined the battle, my squad was at...",2,0,0,1,0
1,71182712,Negative,Sadness,Game is fun and exciting when it works. Very s...,4,0,0,0,1
5,26578229,Positive,Joy,Excellent game. \r\n\r\nPlenty of fun to be ha...,1,1,0,0,0
7,71178247,Negative,Anger,Players are too serious and you can't have fun...,2,0,0,1,0
8,71177738,Positive,Joy,great simulation game,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...
472,70608305,Positive,Joy,the best\r\n,1,1,0,0,0
473,70608216,Positive,Joy,"Most fun when people work together, it is what...",1,1,0,0,0
474,70606979,Negative,Anger,"The game has a lot to learn, but being able to...",2,0,0,1,0
476,70606193,Positive,Joy,"9/10 would recommend, bring friends for 10/10",1,1,0,0,0


# Vectorize DF

In [6]:
vectorizer = CountVectorizer(stop_words='english')

#build features
all_features = vectorizer.fit_transform(df_game_reviews['review'])

# Build Naive Bayes Classifiaction Model 

In [13]:
#split up dataset
X_train, X_test, y_train, y_test = train_test_split(all_features, df_game_reviews.emotion_code, test_size=0.3, random_state=42)

#call model 
classifier = MultinomialNB()

#fit model
classifier.fit(X_train, y_train)

# return count of correctly perdicted docs
print('Naive Bayes Classification Model:')
nr_correct = (y_test == classifier.predict(X_test)).sum()
print(f'{nr_correct} documents classified correctly')

# retrun incorrectly perdicted docs
nr_incorrect = y_test.size - nr_correct
print(f'{nr_incorrect} documents classified incorrectly')

# return percent success
fraction_wrong = nr_incorrect / (nr_correct + nr_incorrect)
print(f'The (testing) accuracy of the model is {1-fraction_wrong:.2%}')

Naive Bayes Classification Model
51 documents classified correctly
37 documents classified incorrectly
The (testing) accuracy of the model is 57.95%


# Build Support Vector Machine and Cross-Validate

In [14]:
#call model
classifier = svm.SVC(gamma=0.001, C=100)

#fit model
classifier.fit(X_train, y_train)

# return count of correctly perdicted docs
print('Support Vector Machine Model:')
nr_correct = (y_test == classifier.predict(X_test)).sum()
print(f'{nr_correct} documents classified correctly')

# retrun incorrectly perdicted docs
nr_incorrect = y_test.size - nr_correct
print(f'{nr_incorrect} documents classified incorrectly')

# return percent success
fraction_wrong = nr_incorrect / (nr_correct + nr_incorrect)
print(f'The (testing) accuracy of the model is {1-fraction_wrong:.2%}')

Support Vector Machine Model:
49 documents classified correctly
39 documents classified incorrectly
The (testing) accuracy of the model is 55.68%
